In [ ]:
# load libraries
import spacy
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

In [ ]:
csv_path = "/content/drive/Shareddrives/NDC_txts/ndcs - sorted with indicators - All indicators (1) (1).csv"
txt_folder = "/content/drive/Shareddrives/NDC_txts"

# Load your CSV
df = pd.read_csv(csv_path)

# Function to read text from file
def read_text(filename):
    file_path = os.path.join(txt_folder, filename)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return None  # or return "" if you prefer an empty string

# Apply the function to the 'filename' column
df["text_content"] = df["filename"].apply(read_text)


NameError: name 'os' is not defined

In [ ]:
df["text_content"]

In [ ]:
!pip install bertopic

  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
Using cached bertopic-0.17.0-py3-none-any.whl (150 kB)


In [ ]:
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import os

In [ ]:
import os
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer

text_folder = "/content/drive/Shareddrives/NDC_txts"
documents = []  # Initialize an empty list to store your documents
for filename in os.listdir(text_folder):
    if filename.endswith(".txt"):
        with open(os.path.join(text_folder, filename), "r", encoding="utf-8") as f:
            documents.append(f.read())  # Append the content of each file to the list

# Now 'documents' contains all the text from the .txt files
# Proceed to generate the document_vectors using your model

# Ensure embeddings are generated from the 'documents' list
model = SentenceTransformer("all-MiniLM-L6-v2")
document_vectors = model.encode(documents, show_progress_bar=True)

# Define UMAP, HDBSCAN, and vectorizer before using them in BERTopic
umap_model = UMAP(n_neighbors=5, n_components=10, min_dist=0.0, metric='cosine', random_state=42)
umap_embeddings = umap_model.fit_transform(document_vectors)
hdbscan_model = HDBSCAN(min_cluster_size=2, metric='euclidean', cluster_selection_method='eom')
cluster_labels = hdbscan_model.fit_predict(umap_embeddings)
vectorizer = CountVectorizer(stop_words='english')

topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    verbose=True
).fit(documents, document_vectors)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

2025-05-07 08:15:12,988 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-07 08:15:13,322 - BERTopic - Dimensionality - Completed ✓
2025-05-07 08:15:13,323 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-07 08:15:13,336 - BERTopic - Cluster - Completed ✓
2025-05-07 08:15:13,362 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-07 08:15:16,096 - BERTopic - Representation - Completed ✓


In [ ]:
# prompt: Can you help me ask it if is able to find any topic words?

# Get the topic info
topic_info = topic_model.get_topic_info()

# Print the topic info DataFrame to see if any topic words are present
topic_info

,Topic,Count,Name,Representation,Representative_Docs
0,-1,17,-1_high_climate_emissions_energy,"[high, climate, emissions, energy, national, c...",[ \n \nLao People's Democratic Republic \nPeac...
1,0,29,0_eu_european_member_regulation,"[eu, european, member, regulation, states, uni...",[ \n \n \n \n1 \n \nUpdate of the \nNDC of the...
2,1,24,1_climate_sector_ndc_emissions,"[climate, sector, ndc, emissions, mitigation, ...",[MINISTRY OF WATER AND ENVIRONMENT\nUpdated Na...
3,2,19,2_climate_change_national_adaptation,"[climate, change, national, adaptation, countr...",[Nationally Determined Contribution\nNationall...
4,3,9,3_oman_energy_water_climate,"[oman, energy, water, climate, national, egypt...",[0 \n \n1 \n \n \n \n2 \n \n \n Executive Sum...
5,4,8,4_uae_climate_barbados_africa,"[uae, climate, barbados, africa, emissions, na...",[Republic of Equatorial Guinea\n MINISTRY OF A...
6,5,8,5_moldova_republic_adaptation_climate,"[moldova, republic, adaptation, climate, devel...","[In accordance with decision No. 1/CP.21, para..."
7,6,7,6_tonga_rwanda_climate_emissions,"[tonga, rwanda, climate, emissions, ndc, jorda...",[ \n \n \n \n \n \n \n \nUPDATED SUBMISSION OF...
8,7,7,7_vanuatu_samoa_seychelles_climate,"[vanuatu, samoa, seychelles, climate, ndc, pal...",[The State of Palestine’s First Nationally\nDe...
9,8,7,8_turkmenistan_türkiye_tajikistan_climate,"[turkmenistan, türkiye, tajikistan, climate, d...","[2 \nThe Republic of Uzbekistan, being committ..."


In [13]:
topic_model.visualize_barchart(top_n_topics=22,n_words=10)